# Scraping Tweets (UK)

This notebook consists the data scraping part of the project, namely, scraping UK tweets from Twitter using the Twitter API.

In [54]:
# Import dependancies
import tweepy
import re
import requests
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import pickle
import time
import itertools

def userids_to_usernames(userids):
    client = tweepy.Client(
        bearer_token="AAAAAAAAAAAAAAAAAAAAAJhtiwEAAAAAnSH7LB8LCFaBffd9NrgflhozHQ8%3Dgl3augKz2Chu0iqikKNcszMsnvQxzPURyiCyQx0DjNwhDDpddO",
        consumer_key="x0ZhcTVGaRmabZ0wzw7cA6TFt",
        consumer_secret="CuI7c9JT8d8PkqkjK73JmTmSiowSQVLpQSAD7KlPwX2DfvKsm7",
        access_token="1454490866597441542-qkkzuAWHrfd84M2DDSBiHQtxjGi6GV",
        access_token_secret="n9X2PFuzNlxfgJdb54Uqkj0SzLIxQDWqW50ZYFZvfdPBi",
    )
    
    response = client.get_users(ids=userids)
    user_names = [user.username for user in response.data]
    errors = [error['value'] for error in response.errors]
    return user_names, errors

In [53]:
# Load data
data_df = pd.read_csv("files/cleaned_twitter_data_UK.csv",index_col=0, dtype = {'user_id': str})

FileNotFoundError: [Errno 2] No such file or directory: 'files/cleaned_twitter_data_UK.csv'

In [46]:
# Inspect first five elements
data_df.head(5)

,tweet_ID,user_id,keywords,tweet_timestamp,valence_intensity,fear_intensity,anger_intensity,happiness_intensity,sadness_intensity,sentiment,emotion,tweet,len_content
0,1.221960e+18,13752142,wuhan,2020-01-28 00:55:15,0.409,0.468,0.530,0.255,0.462,-1.0,A,@maurerbot @JustinTrudeau There's already What...,28.0
1,1.221971e+18,1031691314536935424,wuhan,2020-01-28 01:38:26,0.501,0.473,0.366,0.294,0.367,0.0,NaN,"""Tracking coronavirus: Map, data and timeline""...",16.0
2,1.221976e+18,31473811,wuhan,2020-01-28 01:57:58,0.451,0.453,0.435,0.336,0.451,-1.0,F,@AliEhsassi we have a Canadian baby and her mo...,34.0
3,1.221985e+18,1072771,wuhan,2020-01-28 02:36:25,0.436,0.526,0.415,0.232,0.379,-1.0,F,3 are currently under observation in Quebec fo...,38.0
4,1.221990e+18,401098282,wuhan,2020-01-28 02:53:08,0.501,0.436,0.473,0.351,0.365,0.0,NaN,@Franktmcveety Is this the official site for h...,40.0


In [47]:
data_df = data_df[data_df['user_id'].notna()]

# Inspect shape
data_df.shape

(733090, 13)

In [48]:
# Get list of tweet IDs to scrape
user_IDs = data_df["user_id"].tolist()

user_IDs = list(set(user_IDs))

# Take the first 100 tweets (for testing)
user_IDs_test = user_IDs[:100]
user_array = np.asarray(user_IDs_test)

# Partition into 10 equally sized arrays
split_array = np.array_split(user_array, 10)

for i in range(len(split_array)):
    split_array[i] = list(split_array[i])

In [49]:
# Inspect length
print(len(np.asarray(user_IDs)))

148418


In [50]:
# Get array with tweet IDs
user_array_final = np.asarray(user_IDs)

# Split into equally sized partitions
split_array_final = np.array_split(user_array_final, 1485)

for i in tqdm(range(len(split_array_final))):
    split_array_final[i] = list(split_array_final[i])

# Save list of tweet IDs to pickle file
with open('pickles/list_user_ids_CAN.pkl', 'wb') as f:
    pickle.dump(split_array_final, f)

# Inspect list
print(len(split_array_final))

# Inspect final element
print(len(split_array_final[-1]))


1485
99


In [51]:
print(split_array_final)

[['1022869768284594179', '16900602', '2358055219', '170010382', '273802540', '30992593', '1125758782255128577', '3765449295', '84465948', '188593442', '772195771', '438083101', '719392898', '855007768641757184', '20356391', '1688850228', '352423176', '3426300437', '413025920', '590177602', '3004271428', '1256832385758330880', '865225421981458434', '195342429', '1856015396', '869346218408894465', '361079590', '469668181', '1219816250773245954', '222283378', '853977999477178369', '2445627056', '2753041296', '1266008892', '1117471236', '63506354', '19492746', '1453021129', '358100726', '378853448', '868314030', '1241678043338309645', '564187080', '2704377224', '777720332052344832', '385733347', '262195111', '122738486', '29349453', '102486006', '53944975', '344865382', '45642270', '896018226701848576', '425900856', '1001134932343906309', '28944417', '820450624491884545', '773872913443332096', '114133815', '21160254', '74339196', '46041827', '1055902788', '41497244', '580631181', '15159438

In [52]:
%%time
tweet_list = []
error_list = []

## Retrieve tweets from tweet IDs ##
for i in tqdm(range(len(split_array_final))):
    try:
        tweets, errors = userids_to_usernames(split_array_final[i])   
    except:
        time.sleep(900) # Twitter API only allows for max 900 calls per 15 min. (900 secs) timeframe; sleep and continue
        tweets, errors = userids_to_usernames(split_array_final[i])  

    tweet_list.append(tweets)
    error_list.append(errors)

# Save tweets
with open('pickles/users_CAN.pkl', 'wb') as f:
    pickle.dump(tweet_list, f)

# Save non-existent tweets
with open('pickles/errors_users_CAN.pkl', 'wb') as f:
    pickle.dump(error_list, f)

KeyboardInterrupt: 

In [33]:
# Open tweets list pickle for inspection
with open('files/pickles/tweets_UK.pkl', 'rb') as f:
    mynewlist = pickle.load(f)

# Inspect length
print(len(mynewlist))

7334


In [35]:
# Inspect first tweet
print(mynewlist[0])

['HEALTH EMERGENCY / INTERNATIONAL COVID-19 CORONAVIRUS OUTBREAK: https://t.co/HPvNkqAMH0', 'HEALTH EMERGENCY / INTERNATIONAL COVID-19 CORONAVIRUS OUTBREAK: COVID-19 global cases: Johns Hopkins CSSE online electronic dashboard data as of 2nd March 2020:   https://t.co/QdyloFhT9i', 'Corruption scaring corona virus from Africa &gt;&gt;&gt;', "Starting up with a cold and lightly concerned that it's Corona-chan.\nAlso got a job interview on Thursday, so I'm gonna be spreading it through two major cities.\nI either get the job or you get the Chinese superflu, pick one.", 'The UK government is considering all options, if the coronavirus outbreak continues to escalate. @darrenadam asks: What sacrifices are you willing to accept in combating COVID-19? https://t.co/6oPYBXUoCg', 'USA the land of liberty charges $3000 to get tested for Corona Virus.\nIf this takes off America is as secure as Europe during the black plague. https://t.co/HQChq6ZjJT', 'HEALTH EMERGENCY / COVID-19 CORONAVIRUS OUTBREA

In [38]:
# Check overall number of successfully scraped tweets
final_list = list(itertools.chain.from_iterable(mynewlist))
print(len(final_list))

593237


In [39]:
# Check final tweet
print(final_list[-1])

Israelis protest against Netanyahu, gov't handling of COVID-19 crisis&gt;https://t.co/9Gb7oqkPGs&gt;Brazil’s far-right president, the self-declared homophobe Jair Bolsonaro,https://t.co/uhxYLgZQDL https://t.co/zIVTirnt50
